# **Deep Learning with PyTorch** -  Introductory Lab

## **Part 3:** Deep Learning with PyTorch - Image Denoising Example

* In this part we do a simple deep learning example.
* The aim it to train a Convolutional Neural Network (CNN) for image denoising.

* This notebook is designed to be run on Colab.
* But it can easily be modified to be run locally.

### Setup
* **IMPORTANT!** Activate GPU acceleration by clicking: `Runtime -> Change runtime type -> Hardware Accelerator -> GPU`.
* Unless you already did, you need to make shortcut to the shared data folder in your google drive (see instructions in **part 2** of the lab).
* Run the code in the cell below and follow the instructions to mount your drive to Colab.

In [ ]:
# Connect your google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Set up path to your dataset
# Use this path if you have copied the data to your Google drive
# If you put the data somewhere else, you need to modify this
dataset_path = '/content/gdrive/My Drive/DIV2K_train_small/'

In [ ]:
# Import libraries we will need
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
# Functions we used in part 2

# Read an image with the given name and convert it to torch
def imread(image_file, base_path=None):
    if base_path is None:
        base_path = dataset_path
    im_pil = Image.open(base_path + image_file)
    im_np = np.array(im_pil, copy=False)
    im_torch = torch.from_numpy(im_np).permute(2, 0, 1)
    return im_torch.float()/255

# Show a PyTorch image tensor
def imshow(im, normalize=False):
    # Detatch from graph and move to CPU
    im = im.detach().cpu()

    # Fit the image to the [0, 1] range if normalize is True
    if normalize:
        im = (im - im.min()) / (im.max() - im.min())

    # Remove redundant dimensions 
    im = im.squeeze()    # Mini excersize: check in the documentation what this function does

    is_color = (im.dim() == 3)

    # If there is a color channel dimension, move it to the end
    if is_color:
        im = im.permute(1, 2, 0)

    im_np = im.numpy().clip(0,1)    # Convert to numpy and ensure the values in the range [0, 1]
    if is_color:
        plt.imshow(im_np)
    else:
        plt.imshow(im_np, cmap='gray')
    plt.axis('off')
    plt.show()

### Defining a Neural Network
* Lets first create a CNN that just consists of a single conv layer with no nonlinearities.
* When reading the code, also check the [doc for the Conv2d layer](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#conv2d).

In [ ]:
# In PyTorch, a neural network is represented by a python class.
# All neural networks and network modules must inherit from the nn.Module class.

class LinearConvNet(nn.Module):
    """ A CNN consisting of a single conv layer.
    args:
        kernel_size: Size of the filter. Must be odd.
        bias: Use bias or not.
        init_std: Initialize weights randomly with this standard deviation."""

    # In the constructor we can define the modules and layers that we want the network to have.
    # We should also initialize the weights of the layers.

    def __init__(self, kernel_size=5, num_channels=1):
        super().__init__()   # Always call the constructor of the parent

        # Define a convolutional layer
        # We talked about in/out channels and the kernel size in the previous part of the lab
        # The padding parameter controls the ammount of zeros to implicitly add around the image border.
        # Setting padding=kernel_size//2 will give use the same output size as input size, if kernel_size is odd.
        self.conv = nn.Conv2d(in_channels=num_channels, 
                              out_channels=num_channels, 
                              kernel_size=kernel_size,
                              padding=kernel_size//2)

        # torch.nn.init implements many different initialization strategies.
        # dirac_() simply initializes the filter to the identity mapping.
        torch.nn.init.dirac_(self.conv.weight)


    # In the forward function, you implement what the network module does.
    # In this case, we simply apply the convolutional layer and return its output.

    def forward(self, im):
        return self.conv(im)


* In the forward function above, note that `self.conv(im)` is essentially equivalent to the functional form we used in the previous part: `F.conv2d(im, self.conv.weight)`
* The `self.conv` object internally stores the convolution weights we want to learn, along with the other settings of the convolution (such as the padding).
* We can therefore apply it to an input image with the simple call `self.conv(im)`.


### Applying a Neural Network
* Lets apply the network to an image.
* First we need to create an instance of the network.

In [ ]:
# Create a network object
net = LinearConvNet(kernel_size=9, num_channels=1)

In [ ]:
# Load an example image as in the previous part
im = imread('0705x4.png')
im_gray = im.mean(dim=0)
imshow(im_gray)

In [ ]:
# Run the network on the image
# This will call the forward function of the network automatically
# As before, we first need to resize the image to be 4D

im_out = net(im_gray.view(1, 1, *im_gray.shape))

imshow(im_out)

* Note that the output of the network is identical to the input.
* That is because we initialize the only convolutional layer to be identity (a dirac).
* We can manually change the weights to an averaging filter and see the effect ...

In [ ]:
# change the weights to an averaging filter
net.conv.weight[:] = 1 / net.conv.weight.nelement()

im_out = net(im_gray.view(1, 1, *im_gray.shape))

imshow(im_out)

### Towards Deep Image Denoising
* The goal in the next steps is to train the network to do image denoising.

#### Generating Noise
* To start with, we need a function that can generate noisy images from clean ones.

In [ ]:
# Lets make a simple function that adds noise.
def add_noise(im, std=0.1):
    return im + std * torch.randn(im.shape)

In [ ]:
# Lets test it
im_noisy = add_noise(im)
imshow(im_noisy)

#### Objective function
* To train the network, we need an objective function (loss) that measures the difference between the network output and the clean reference image.
* Lets use the Mean Squared Error (i.e. L2 loss).
* You can check the [doc here](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html?highlight=mse#torch.nn.MSELoss).

In [ ]:
# The objectively is also fundamentally a neural network module
objective = nn.MSELoss()

In [ ]:
# Lets test our objective
# First we create a network designed for taking RGB images as input

net = LinearConvNet(kernel_size=9, num_channels=3)

In [ ]:
# Now, lets compute the loss between the network output and the clean image

# Make image 4d
im_reference = im.view(1, *im.shape)

# Add noise for the input
im_input = add_noise(im_reference)

# Apply network
im_output = net(im_input)

# Compute loss
loss = objective(im_output, im_reference)

print(loss)

* The value represents the Mean Squared Error over all pixels

#### Computing Gradients
* To train the network with Stochastic Gradient Descent (see lecture), we first need to compute the gradients of the objective function w.r.t. the weights we want to optimize.
* Fortunately, this is extremely easy in PyTorch.
* We just need to call the `backward()` function on the computed loss.

In [ ]:
# Compute gradients
loss.backward()

In [ ]:
# The gradients are stored in the .grad attribute of all weights in the network
# Lets print some info:
print('grad shape:', net.conv.weight.grad.shape)
print('grad mean:', net.conv.weight.grad.mean())
print('grad min:', net.conv.weight.grad.min())
print('grad max:', net.conv.weight.grad.max())

* To perform a gradient descent step, we could essentially do the following
```
for w in net.parameters():
      w += learning_rate * w.grad
```
* Here learning_rate is the size of the gradient step.
* However, PyTorch already have convenient oprimizers implemented.
* Lets check how it works next.

#### The Optimizer
* We first define an optimizer.
* Lets choose the standard SGD ([see doc](https://pytorch.org/docs/stable/optim.html?highlight=sgd#torch.optim.SGD)).

In [ ]:
# First define the network again
net = LinearConvNet(kernel_size=9, num_channels=3)

# To the optimizer, we specify the parameters we whish to optimize and the learning rate (lr)
# Here, we also set the momentum parameter, which also averages gradients over time
optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [ ]:
# Now, lets create a loop that optimizes the network on a single image

im = imread('0705x4.png')

num_steps = 2000

for n in range(num_steps):
    # Make image 4d
    im_reference = im.view(1, *im.shape)

    # Add noise for the input
    im_input = add_noise(im_reference)

    # We need to clear the old gradients first
    optimizer.zero_grad()

    # Apply network
    im_output = net(im_input)

    # Compute loss
    loss = objective(im_output, im_reference)

    # Compute the new gradients
    loss.backward()

    # Take an optimization step
    optimizer.step()

    # Print loss to show progress
    if n % 50 == 0:
         print('Iter {}/{}:  loss = {}'.format(n, num_steps, loss.item()))

In [ ]:
# Compare the last input and output of the network
imshow(im_input[...,:100,100:200])
imshow(im_output[...,:100,100:200])

* The network has learned to donoise the image to some extent!
* We saw that the loss was decreasing quite rapidly over the iteration
### 💡 **Exercise**
* Apply the trained network on another image and check the results visually. What do you think?

In [ ]:
# Implement your solution here


#### Creating a dataset loader

* Note that we only trained the network above on a single image.
* In general, this would not work well in practice, especially if using deeper networks.
* The network would just overfit to that image and not generalize to other images.
* Here, we will implement training on a full dataset using standard PyTorch tools.
* First we will copy the data over to the colab instance.

In [ ]:
# Make a directory called data
!mkdir /content/data 

In [ ]:
# Copy the images for training and testing
# This takes a while ... read and understand the next cells in the meantime!
!rsync -avzh /content/gdrive/My\ Drive/DIV2K_train_small/* /content/data/

In [ ]:
from pathlib import Path
from torchvision import transforms     # Torchvision is a part of PyTorch that contains extra vision functionality

class DenoisingDataset(torch.utils.data.Dataset):
    def __init__(self, dir, noise_std=0.1, crop_size=200, pattern='*.png', start_ind=None, end_ind=None):
        super().__init__()

        self.noise_std = noise_std
        self.crop_size = crop_size

        # List all images in the folder
        image_dir = Path(dir)
        self.image_files = list(sorted(image_dir.glob(pattern)))
        self.image_files = self.image_files[start_ind:end_ind]

        # Create transform that crops the image and converts to tensor
        self.transform = transforms.Compose([transforms.RandomCrop(size=self.crop_size, pad_if_needed=True),
                                             transforms.ToTensor()])


    # The dataset need to implement the getitem function. This should load and return an image pair.
    # This function is called with a random index during training.
    def __getitem__(self, index):
        # Get image path
        im_path = self.image_files[index]

        # Load the PIL image
        im_pil = Image.open(im_path)
        
        # Transform to tensor. This also do normalization
        im_reference = self.transform(im_pil)

        # Add noise
        im_noisy = add_noise(im_reference, std=self.noise_std)

        return im_reference, im_noisy

    # The length function should return the number of images in the dataset
    def __len__(self):
        return len(self.image_files)

In [ ]:
# Lets test the dataset loader.
dataset_train = DenoisingDataset(dir='/content/data/')

im_ref, im_input = dataset_train[4]

imshow(im_ref)
imshow(im_input)


In [ ]:
# Now we need to create a data loader. This is easy
# batch_size is the number of images we sample in parallell.
# The loader will automatically sample images from the dataset and concatenate them

train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=32, shuffle=True, drop_last=True)

In [ ]:
# Lets test it!
for im_ref, im_input in train_loader:
    print('ref shape:', im_ref.shape)
    print('input shape:', im_input.shape)

* Note that the tensors returned by the loader consists of 32 stacked color images of size 200x200.
* 32 is the batch_size we chose in the previous cell.
* The loop above goes through all images in the dataset once.

#### The training loop

* We saw earlier a simple example of a training loop for a single image.
* Now we will write the training loop for the real case.
* This will implement real Stochastic Gradient Descent, since also the data is sampled as random batches during training.
* The training is usually split into **epochs**.
* Each **epoch** cycles through the dataset once. 

In [ ]:
# We first write a function that runs a single epoch

def run_epoch(net, objective, optimizer, data_loader, is_training=True, cuda=False):
    # If we are training on GPU, make sure that the network is there
    if cuda:
        net.cuda()
    else:
        net.cpu()

    # Set network to train/val mode
    net.train(is_training)

    # Turn of tracking of gradients if we are doing validation (saves memory)
    torch.set_grad_enabled(is_training)

    # For tracking the average loss
    avg_loss = 0
    num_batches = 0

    # Loop over the dataset
    for im_reference, im_input in data_loader:
        # Move data to GPU if we use it
        if cuda:
            im_reference = im_reference.cuda()
            im_input = im_input.cuda()
        
        # Reset gradients
        if is_training:
            optimizer.zero_grad()
        
        # Apply network
        im_output = net(im_input)

        # Compute loss
        loss = objective(im_output, im_reference)

        if is_training:
            # Compute gradients
            loss.backward()

            # Update weights by taking one optimizer step
            optimizer.step()
        
        # Update statistics
        avg_loss += loss.item()
        num_batches += 1

    avg_loss /= num_batches

    # Return average loss so that we can print that
    return avg_loss


* In each epoch, we should train the network on the training set.
* We should also evaluate the network at each epoch.
* This is very important in order to monitor the training and making sure that the network does not overfit on the training data.

In [ ]:
# Now we write the full training loop
# It should do both training and validation

def train_network(net, objective, optimizer, train_loader, val_loader, num_epochs, cuda=False):
    # Loop over all epochs
    for ep in range(num_epochs):
        # Run the training
        train_loss = run_epoch(net, objective, optimizer, train_loader, is_training=True, cuda=cuda)

        # Run the validation
        val_loss = run_epoch(net, objective, optimizer, val_loader, is_training=False, cuda=cuda)

        # Print stats
        print('[Ep {}/{}] Train loss: {:.06f}  ,  Val loss: {:.06f}'.format(ep+1, num_epochs, train_loss, val_loss))

    print('Training done!')


#### Running a first training
* Now we have all the tools to run our first real training.
* Lets set it up!

In [ ]:
# Lets first specify some general settings
cuda = True         # Run with GPU on since that will be faster.
batch_size = 32

# Define the training dataset. The final 50 images (end_ind=-50) we can save them for validation
dataset_train = DenoisingDataset(dir='/content/data/', end_ind=-50)

# Similarily we define the validation dataset
dataset_val = DenoisingDataset(dir='/content/data/', start_ind=-50)

# Create the dataset loaders
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size, shuffle=True, drop_last=True)

# Create the network 
net_linear = LinearConvNet(kernel_size=9, num_channels=3)

# Lets use the same objective
objective = nn.MSELoss()

# Also the same SGD optimizer as before
optimizer = torch.optim.SGD(net_linear.parameters(), lr=0.05, momentum=0.9)

In [ ]:
# Lets start the training!
train_network(net_linear, objective, optimizer, train_loader, val_loader, num_epochs=50, cuda=cuda)

* Now we have trained our first network. Lets test it on an image of the validation set.

In [ ]:
# We should move the network back to CPU
net_linear.cpu()
net_linear.eval()

# Read an image from the validation set
im_ref = imread(str(dataset_val.image_files[0]), base_path='')

im_noisy = add_noise(im_ref)

im_denoised = net_linear(im_noisy.unsqueeze(0))

imshow(im_noisy[..., 100:200, :100])
imshow(im_denoised[..., 100:200, :100])

### Creating a deep network
* The network we have trained so far consists of just a single convolution.
* Now we will build our first deep network.
* The code below implements the CNN architecture in this figure.
* Note that the second convolution downsamples the feature map by a factor 2 and that the third one up-samples it again.
![could not be displayed](https://raw.githubusercontent.com/martin-danelljan/pytorch-tutorial/master/images/miniunet.png)

In [ ]:

class MiniUNet(nn.Module):
    """A small U-Net architecture with skip connections."""

    def __init__(self, bias=True, init_std=1e-2):
        super().__init__()

        # Initial conv that preserves the resolution
        self.conv_init = nn.Conv2d(3, 16, kernel_size=7, stride=1, padding=3, bias=bias)

        # A conv layer that also downsamples the data (stride=2)
        self.conv1 = nn.Conv2d(16, 32, kernel_size=5, stride=2, padding=2, bias=bias)

        # One conv layer that upsample the data again.
        # These type of layers are called transposed convolutions.
        self.convtp1 = nn.ConvTranspose2d(32, 16, kernel_size=6, stride=2, padding=2, bias=bias)

        # Add a final conv layer that generates the output
        self.conv_final = nn.Conv2d(16, 3, kernel_size=3, stride=1, padding=1, bias=bias)

        # We use the rectified linear unit activation
        self.activation = nn.ReLU(inplace=True)

        # Initialization
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
                # We randomly initialize the weights with some standard deviation
                torch.nn.init.normal_(m.weight, std=init_std)
                if m.bias is not None:
                    torch.nn.init.zeros_(m.bias)


    def forward(self, im):
        # Layer 1: (same resolution)
        x1 = self.activation(self.conv_init(im))

        # Layer 2: (2x down sampling)
        x2 = self.activation(self.conv1(x1))

        # Layer 3: (2x up sampling)
        x3 = self.activation(self.convtp1(x2))

        # Layer 4: (same as initial resolution)
        im_out = self.conv_final(x3)

        return im_out

In [ ]:
# Lets first specify some general settings
cuda = True         # Run with GPU on since that will be faster.
batch_size = 32

# Define the training dataset. The final 50 images (end_ind=-50) we can save them for validation
dataset_train = DenoisingDataset(dir='/content/data/', end_ind=-50)

# Similarily we define the validation dataset
dataset_val = DenoisingDataset(dir='/content/data/', start_ind=-50)

# Create the dataset loaders
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size, shuffle=True, drop_last=True)

# Create the network 
miniunet = MiniUNet()

# Lets use the same objective
objective = nn.MSELoss()

# Also the same SGD optimizer as before
optimizer = torch.optim.SGD(miniunet.parameters(), lr=0.05, momentum=0.9)

In [ ]:
# Lets start the training!
train_network(miniunet, objective, optimizer, train_loader, val_loader, num_epochs=50, cuda=cuda)

* We see that the network starts with a very high loss compared to that of the linear network.
* Moreover, it converges very slowly.
* Lets try to address that.
### 💡 **Exercise**
* Add residual connections (also known as skip connections) to the MiniUNet architecture according to the following figure.
* "+" denotes standard pointwise addition.
* Note that the long residual connection adds the noisy input image to the output.
* This means that the network will predict the noise, which is then substracted from the input image.
![could not be displayed](https://raw.githubusercontent.com/martin-danelljan/pytorch-tutorial/master/images/miniunet_residual.png)

In [ ]:
# You can modify the code above or make a new copy of it in another cell
# Then train you new network architecture here


### 💡 **Exercise**
* Although the loss starts at a reasonable value, the convergence is still slow.
* In CNNs, this is often addressed using Batch Normalization layers.
* Thease layers often makes deep networks significantly easier and faster to train.
* Check the [PyTorch documentation for the BatchNorm2d layer](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html?highlight=batchnorm#torch.nn.BatchNorm2d).
* Then add Batch Normalization (BN) layers according to the figure below.
* **Tip:** You can keep the optinal parameters to their default values.
![could not be displayed](https://raw.githubusercontent.com/martin-danelljan/pytorch-tutorial/master/images/miniunet_bn.png)

In [ ]:
# Train your new architecture here


### 💡 **Exercise**
* Visually check the denoising quality of your network using the code below.
* Check for different images and different regions in the image.
* What do you think about the quality? How does it compare with the linear network? 

In [ ]:
# We should move the network back to CPU
miniunet.cpu()

# The network should be in eval() mode. This is very important when using Batch Norm.
miniunet.eval()

# Read an image from the validation set
im_ref = imread(str(dataset_val.image_files[0]), base_path='')

# We need to select a crop that is divisible with 4
im_ref = im_ref[:, :256, :256]

im_noisy = add_noise(im_ref)

im_denoised = miniunet(im_noisy.unsqueeze(0))

imshow(im_noisy[..., 100:200, :100])
imshow(im_denoised[..., 100:200, :100])

Here are some extra exercises you can do if you get time. or after the lab if you are interested.

### 💡 **Exercise (extra)**

* Try applying the network to other images with other noise standard deviations.
* How are the results?

### 💡 **Exercise (extra)**

* To improve the performance of the network for other noise distributions, you can try to randomly sample the noise standard deviation during training.
* Add an option in the `DenoisingDataset` for this.
* Note that the `add_noise` function has an optinal parameter for the standard deviation.

### 💡 **Exercise (extra)**

* Try making the network deeper by adding additional layers in the middle.

### 💡 **Exercise (extra)**

* Replace SGD with the more recent [Adam optimizer](https://pytorch.org/docs/stable/optim.html?highlight=adam#torch.optim.Adam).

### 💡 **Exercise (extra)**

* Try changing to a [Leaky Relu activation](https://pytorch.org/docs/stable/generated/torch.nn.LeakyReLU.html?highlight=leaky%20relu#torch.nn.LeakyReLU).

### 💡 **Exercise (extra)**

* Integrate a learning rate scheduler that decreases the learning rate for each epoch.
* Use, for example, the [StepLR](https://pytorch.org/docs/stable/optim.html?highlight=schedule#torch.optim.lr_scheduler.StepLR). 
